In [78]:
import numpy as np
import sympy as sym
import math
import matplotlib.pyplot as plt

In [79]:
x = sym.Symbol('x',real=True)

In [80]:
def GetLaguerre (n,x):
    
    if n == 0:
        poli = sym.Number(1)
    elif n ==1:
        poli = (1-x)
    else:
        poli = ((2*n-1-x)*GetLaguerre(n-1,x)-(n-1)*GetLaguerre(n-2,x))/n
        
    return sym.expand(poli,x)

In [81]:
GetLaguerre(3,x)

-x**3/6 + 3*x**2/2 - 3*x + 1

In [82]:
def derivadasLaguerre(n,x):
    
    derivadas = GetLaguerre(n,x)
    
    return sym.diff(derivadas,x,1)

In [83]:
def GetNewton(f,df,xn,itmax=10000,precision=1e-14):
    
    error = 1.
    it = 0
    
    while error >= precision and it < itmax:
        
        try:
            
            xn1 = xn - f(xn)/df(xn)
            
            error = np.abs(f(xn)/df(xn))
            
        except ZeroDivisionError:
            print('Zero Division')
            
        xn = xn1
        it += 1
        
    if it == itmax:
        return False
    else:
        return xn

In [84]:
def GetRoots(f,df,x,tolerancia = 10):
    
    Roots = np.array([])
    
    for i in x:
        
        root = GetNewton(f,df,i)

        if  type(root)!=bool:
            croot = np.round( root, tolerancia )
            
            if croot not in Roots:
                Roots = np.append(Roots, croot)
                
    Roots.sort()
    
    return Roots

In [85]:
def GetAllRootsGLag(n):
    
    vn = np.linspace(0,n+(n-1)*np.sqrt(n),100)
    
    Laguerre = []
    D_Laguerre = []
    
    for i in range(n+1):
        Laguerre.append(GetLaguerre(i,x))
        D_Laguerre.append(derivadasLaguerre(i,x))
    
    poli = sym.lambdify([x],Laguerre[n],'numpy')
    D_poli = sym.lambdify([x],D_Laguerre[n],'numpy')
    Roots = GetRoots(poli,D_poli,vn)

    if len(Roots) != n:
        ValueError('El número de raíces debe ser igual al n del polinomio.')
    
    return Roots

In [86]:
raices = GetAllRootsGLag(3)

In [87]:
def GetWeightsGLag(n):

    Roots = GetAllRootsGLag(n)

    Laguerre = []
    
    for i in range(n+2):
        Laguerre.append(GetLaguerre(i,x))
    
    poli = sym.lambdify([x],Laguerre[n+1],'numpy')
    Weights = Roots/(((n+1)**2)*(poli(Roots)**2))
    
    return Weights

In [88]:
pesos = GetWeightsGLag(3)

In [89]:
f = lambda x: (x**3)/(np.exp(x)-1)

In [90]:
def integral(n):

    I = 0
    for i in range(3):
        I += pesos[i]*f(raices[i])*np.exp(raices[i])
    
    return I

In [91]:
I

6.481130171427754

In [92]:
"Parte b"

'Parte b'

In [93]:
teorico = (np.pi**4)/15

N = np.linspace(2,10,9)

exactitud = []

for i in N:
    
    experimental = integral(i)
    
    error = experimental/teorico
    
    exactitud.append(error)
    
